In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests


In [24]:
def scrape(path):
    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"E:\web_scarping\project_3\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)

    # details
    # details = {}
    results = []

    # =============================================================================================
    # complete login

    try:
        provided_u_name = "haider1805@icloud.com"
        user_name_id = "username"
        user_name = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.NAME, user_name_id)))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print("No username tab found")
    # get password tab
    try:
        provided_pass = "Muhssan7865"
        pass_id = "password"
        password = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.NAME, pass_id)))
        password.send_keys(provided_pass)
    except Exception as e:
        print("No password tab found") 
     # get the login tab to enter the username and password
    try:
        # login_tab_css = "submit"
        login_tab = driver.find_element(By.CLASS_NAME,  "onelogin-main__btn")
        driver.execute_script("arguments[0].scrollIntoView();", login_tab)
        login_tab.click()
    except Exception as e:
        print("No login tab found here")
    
    # =====================================================================================  
    # get the number of cars found
    try:
        num_cars_css = "//p[@data-testid='lot-navigation-number']" # here we will have number of cars found for the current website
        num_cars = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, num_cars_css)))
        nums = num_cars.text
        match = re.search(r"/(\d+)", nums) # beacuse of the date format
        total_cars = match.group(1)
    except Exception as e:
        print("Nothing found")
    
    # =============================================================================================
    # Handle cookie consent banner
    try:
        # cookie_main_bar = driver.find_element(By.ID, "onetrust-button-group")
        cookie_accept = driver.find_element(By.ID, "onetrust-reject-all-handler")
        driver.execute_script("arguments[0].scrollIntoView(true);", cookie_accept)
        time.sleep(1)
        cookie_accept.click()
    except Exception as e:
        print(f"Error handling cookie banner: {e}")

    # =====================================================================================  
    # # get the next button
    # try:
    #     # next_button_bar = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, "OVeuM")))
    #     next_button = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="next-lot-navigation-button"]')))
    #     # driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
    #     # next_button = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a.hpQdrD")))
    #     # print(next_button.text)

    # except Exception as e:
    #     print("No next button here")

    # =====================================================================================  

    # run till the number of cars available from total_cars above
    for i in range(5): # for sample purpose
        details={}
        # try:
        #     # cookie_main_bar = driver.find_element(By.ID, "onetrust-button-group")
        #     cookie_accept = driver.find_element(By.ID, "onetrust-reject-all-handler")
        #     driver.execute_script("arguments[0].scrollIntoView(true);", cookie_accept)
        #     cookie_accept.click()
        # except Exception as e:
        #     print(f"Error handling cookie banner: {e}")
        # get the car reg and name, mileage, featured in, planned, date & venue, auction name, vehicle location and lot number
        try:
            container_class = "VehicleInteraction__VehicleDetailsWrapper-sc-1g915f-0"
            container = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, container_class)))

            # get the reg
            reg_class = "bLafSa"
            reg = container.find_element(By.CLASS_NAME, reg_class)

            # get the car name
            name_css = "VehicleInteraction___StyledTypography-sc-1g915f-19"
            name = container.find_element(By.CLASS_NAME, name_css)

            # get the mileage
            mileage = driver.find_element(By.CSS_SELECTOR, "h6.sc-fqkvVR.iUxEsY")
            mileage_text=mileage.text 

            # get the featured in as a label
            feat_in_l = driver.find_element(By.CSS_SELECTOR, "p.jLnlNa").text
            # get the value of above
            feat_in_v = driver.find_element(By.CSS_SELECTOR, "h5.sc-fqkvVR.kMeqUD").text

            # get the planned label
            planned_l = driver.find_element(By.CSS_SELECTOR, "span.iXOsDU").text
            planned_v = driver.find_element(By.CSS_SELECTOR, "p.sc-fqkvVR.jPRetG").text

            # get the day-date and venue
            date_ven_lst = []
            date_ven = driver.find_elements(By.CSS_SELECTOR, "p.sc-fqkvVR.bnHyrd")
            for v in date_ven:
                date_ven_lst.append(v.text)

            # get auction name
            auction_name = driver.find_element(By.CSS_SELECTOR, "h6.EeoBD").text
            # get location
            veh_location = driver.find_element(By.CLASS_NAME, "fUnarQ").text
            # get lots
            lot_num = driver.find_element(By.CLASS_NAME, "joByeZ").text


            details["Reg_No"] = reg.text
            details['Name'] = name.text
            details["Mileage"] = mileage_text
            details[feat_in_l] = feat_in_v
            details[planned_l] = planned_v
            details['Date_venue'] = date_ven_lst
            details['Auction_name'] = auction_name
            details['Vehicle_Location'] = veh_location
            details['Lot_number'] = lot_num

        except Exception as e:
            print("No reg and name found")

    # =====================================================================================  
        # get the car images
        try:
            images = []
            img_cont_class = "Thumbnails__ImagesContainer-sc-tbif6y-0"
            img_cont = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CLASS_NAME, img_cont_class)))

            # get the images now
            imgs = img_cont.find_elements(By.TAG_NAME, "img")
            for img in imgs:
                images.append(img.get_attribute("src"))
            imgs_str = ", ".join(images)
            details['Images'] = imgs_str
        except Exception as e:
            print("No images found")

    # =====================================================================================  
        # get the SPECIFICATION
        try:
            main_pg = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".cDPOzx")))
            
            labels_css = ".cCprlp"
            # divs = WebDriverWait(driver, 60).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, divs_css)))
            labels = main_pg.find_elements(By.CSS_SELECTOR, labels_css)
            labels_list = []

            # get all labels
            for label in labels:
                labels_list.append(label.text)

            values_css = ".khatBv"
            values = main_pg.find_elements(By.CSS_SELECTOR, values_css)
            values_list = []

            #get all the values
            for value in values:
                values_list.append(value.text)
            
            # append the labels and values in the dictionary
            temp_dict=dict(zip(labels_list, values_list))
            # update the existance dictionary
            details.update(temp_dict)
        except Exception as e:
            print("No divs")

    # ===================================================================================== 
        # get the decleration
        try:
            dec_main = driver.find_element(By.CSS_SELECTOR, "div.dRCqKq")
            dec_label = dec_main.find_element(By.CSS_SELECTOR, "h5.hRqhbX").text
            dec_values = dec_main.find_elements(By.CSS_SELECTOR, "p.gHHIJJ")
            if len(dec_values) >= 1:
                dec_list = []
                for dec_v in dec_values:
                    dec_list.append(dec_v.text)
                details[dec_label] = dec_list
            else:
                details[dec_label] = "No decelartions found"

        except Exception as e:
            details["Declerations"] = "No decelartions found"
    # =====================================================================================
        # get the equipment and additional information
        try:
            # Locate all sections with the same class
            main_sections = driver.find_elements(By.CSS_SELECTOR, "div.CATTm")

            # Loop through all sections to identify and extract the correct data
            for section in main_sections:
                try:
                    # Get the label of the section, regardless of its CSS class
                    section_label = section.find_element(By.CSS_SELECTOR, "h5").text.lower()

                except Exception as e:
                    print("Section label not found:", e)
                    continue  # Skip this section if label is missing

                # Identify the equipment section
                if "equipment" in section_label:
                    try:
                        eq_header = section.find_element(By.CSS_SELECTOR, "div.eVCHPp")
                        eq_values = eq_header.find_elements(By.TAG_NAME, "li")

                        if len(eq_values) > 0:
                            eq_list = [eq_value.text for eq_value in eq_values]
                            details["Equipment"] = eq_list
                        else:
                            details["Equipment"] = "No equipment found"
                    except Exception as e:
                        print("Error extracting equipment details:", e)               

                # Identify the additional information section
                elif "additional information" in section_label:
                    try:
                        add_header = section.find_element(By.CSS_SELECTOR, "div.eVCHPp")
                        add_values = add_header.find_elements(By.TAG_NAME, "li")

                        if len(add_values) > 0:
                            add_list = [add_value.text for add_value in add_values]
                            details["Additional Information"] = add_list
                        else:
                            details["Additional Information"] = "No additional information found"
                    except Exception as e:
                        print("Error extracting additional information details:", e)

        except Exception as e:
            print("Error occurred:", e)

    # =====================================================================================
        # get the conditon report
        try:
            cond_sec = driver.find_element(By.CSS_SELECTOR, "div.kjlvHR")
            grade_label = cond_sec.find_element(By.CSS_SELECTOR, "div.YpBMK").text
            grade_val = cond_sec.find_element(By.CSS_SELECTOR, "div.frxkxC").text
            details[grade_label] = grade_val

            # get the text
            cond_text = cond_sec.find_element(By.CSS_SELECTOR, "p.fBEwrJ")
            details['Condition_Report_text'] = cond_text.text

            # get the link
            cond_rep = cond_sec.find_element(By.XPATH, "//a[text()='View Condition Report']")
            report = cond_rep.get_attribute('href')
            details["Condition_report"] =  report

        except Exception as e:
            details["Condition report"] = "No condition report found"

    # ===================================================================================== 
        # get the conditon report
        try:
            assured_sec = driver.find_element(By.CSS_SELECTOR, "div.eqmWdV")
            # get the text
            assured_text = assured_sec.find_element(By.CSS_SELECTOR, "p.iYoHiO")
            details['Condition_Report_text'] = assured_text.text

            # get the link
            assured_rep = assured_sec.find_element(By.XPATH, "//a[text()='View Assured Report']")
            report = assured_rep.get_attribute('href')
            details["Assured_report"] =  report

        except Exception as e:
            details["Condition report"] = "No condition report found"

    # =====================================================================================
        try:
            # Locate the main base div with class XEQhH
            base_div = driver.find_element(By.CSS_SELECTOR, "div.XEQhH")
            # get the CAP HPI
            try:
                cap_hpi_sec = base_div.find_element(By.CSS_SELECTOR, "div.iUYhNI")
                cap_hpi = cap_hpi_sec.find_element(By.CSS_SELECTOR, "h6.fkQpmB").text

                # get all the column names 
                cols = cap_hpi_sec.find_elements(By.CSS_SELECTOR, ".hcEGbM")
                cols_list = []
                for col in cols:
                    cols_list.append(col.text)

                # get all values
                vals = cap_hpi_sec.find_elements(By.CSS_SELECTOR, ".bwOa-dQ")
                val_list=[]
                for val in vals:
                    val_list.append(val.text)
                temp_dict_2 = dict(zip(cols_list, val_list))
                details[cap_hpi] = temp_dict_2
            except Exception as e:
                details['CAP HPI'] = "no details found for cap hpi pricing"
            # ===================================================================
            try:
                # get the Glass's
                glass_sec = base_div.find_element(By.CSS_SELECTOR, "div.ShvzP")
                glass = glass_sec.find_element(By.CSS_SELECTOR, "h6.kKa-DOe").text

                # get all the column names 
                g_cols = glass_sec.find_elements(By.CSS_SELECTOR, ".hcEGbM")
                g_cols_list = []
                for g_col in g_cols:
                    g_cols_list.append(g_col.text)

                # get all values
                g_vals = glass_sec.find_elements(By.CSS_SELECTOR, ".bwOa-dQ")
                g_val_list=[]
                for g_val in g_vals:
                    g_val_list.append(g_val.text)
                g_temp_dict_2 = dict(zip(g_cols_list, g_val_list))
                details[glass] = g_temp_dict_2
            except Exception as e:
                details['Glass'] = "no details found for glass pricing"
            # ===================================================================================
            try:
                get_text = base_div.find_element(By.CSS_SELECTOR, "h6.iGcAgx")
                details['Pricing_text'] = get_text.text
            except Exception as e:
                details['Pricing_text'] = "no texts of pricing found"
            # =====================================================================================

        except Exception as e:
            print("Error occurred:", e)
    # =====================================================================================

        results.append(details)
        next_button = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="next-lot-navigation-button"]')))
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        next_button.click()
        i+=1
    df = pd.DataFrame.from_dict(results)
    df.to_csv("bca.csv")
    driver.quit()
path = "https://www.bca.co.uk/lot/RE67%20HXA?q=&bq=VehicleType%3ACars&searchVersion=new"
scrape(path)

In [2]:
df = pd.read_csv("bca.csv")
df.columns

Index(['Unnamed: 0', 'Reg_No', 'Name', 'Mileage', 'Featured in', 'Confirmed',
       'Date_venue', 'Auction_name', 'Vehicle_Location', 'Lot_number',
       'Images', 'Vehicle Type', 'Colour', 'Fuel', 'Transmission',
       'No. of doors', 'CO2 Emissions', 'NOX Emissions', 'No. of keys',
       'Log book', 'No. of owners', 'Date of registration', 'VAT Type',
       'Service History', 'Number of services', 'Last Service',
       'Last service mileage', 'DVSA mileage', 'Additional service notes',
       'MOT Expiry', 'Declarations', 'Equipment', 'Grade',
       'Condition_Report_text', 'Condition_report', 'Assured_report',
       'CAP HPI*', 'Glass's', 'Pricing_text', 'Condition report',
       'Declerations', 'Planned'],
      dtype='object')

# Now download the images from the links in the above df

In [3]:
# separate the reg_no and images columns from the above dataframe
reg_img = df[['Reg_No', 'Images']]
reg_img['Images'][0]

'https://www1.bcaimage.com/Document?DocType=VehicleImage&Reg=RE67HXA-GB&grp=public&obl=hht3,Manheim&minwidth=600&width=600&default=5, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395278, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395281, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395285, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395290, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395296, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395303, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395315, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395318, https://www1.bcaimage.com/Document?DocType=VehicleImage&width=600&docId=561395323, https://www1.bcaimage.com/Document?DocType=VehicleImage&docId=561288154, https://www1.bcaimage.com/Document?DocType=Veh

In [4]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg_No"] # separate reg nums
        image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # loop around the urls of the current row and also save index for further use
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                url = urljoin("https://", url) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(url)

            # check if the parsed url is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                continue
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(url, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Extracts the file extension (e.g., jpg). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")
# Call the function
download_images(reg_img)


Downloaded: Images\RE67 HXA\RE67 HXA_1_image_1.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_2_image_2.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_3_image_3.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_4_image_4.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_5_image_5.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_6_image_6.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_7_image_7.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_8_image_8.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_9_image_9.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_10_image_10.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_11_image_11.jpg
Downloaded: Images\RE67 HXA\RE67 HXA_12_image_12.jpg
Downloaded: Images\LK20 UOF\LK20 UOF_1_image_1.jpg
Downloaded: Images\LK20 UOF\LK20 UOF_2_image_2.jpg
Downloaded: Images\LK20 UOF\LK20 UOF_3_image_3.jpg
Downloaded: Images\LK20 UOF\LK20 UOF_4_image_4.jpg
Downloaded: Images\LK20 UOF\LK20 UOF_5_image_5.jpg
Downloaded: Images\LK20 UOF\LK20 UOF_6_image_6.jpg
Downloaded: Images\LK20 UOF\LK20 UOF_7_image_7.jpg
Downloaded: Images\LK20 U